# 17 - Instrumental Variable Analysis

Marina Adshade, Paul Corcuera, Giulia Lo Forte, Jane Platt  
2024-05-29

## Prerequisites

1.  Run OLS regressions.

## Learning Outcomes

1.  Understand what an instrumental variable is and the conditions that
    must be satisfied to address the endogeneity problem.
2.  Implement a Two Stage Least Squares (2SLS) regression-based approach
    using an instrument.
3.  Describe the weak instrument problem.
4.  Interpret the first stage test of whether or not the instrument is
    weak.

## 17.1 The Linear Instrumental Variable Model

Consider a case where we want to know the effect of education on
earnings. We may want to estimate a model like the following:

$$
Y_{i} = \alpha + \beta X_i + \epsilon_i,
$$

where $Y_i$ is earnings of individual $i$ and $X_i$ is years of
education of individual $i$.

A possible issue with this model comes from omitted variable bias: it is
possible that the decision to attend school is influenced by other
individual characteristics that are also correlated with earnings. For
example, think of individuals with high innate ability. They may want to
enroll in school for longer and obtain higher-level degrees. Moreover,
their employers may compensate them for their high ability, regardless
of their years of schooling.

Instrumental variables (IVs) can help us when there are hidden factors
affecting both the treatment (in our case, years of education) and the
outcome (in our case, earnings). The instrumental variable approach
relies on finding something that affects the treatment and affects the
outcome, but that affects the outcome solely through the treatment. In
short, the instrument must satisfy two assumptions:

1.  *Relevance*: the instrument should be correlated with the
    explanatory variable; in our case, it should be correlated with the
    years of education $X_i$;
2.  *Exclusion restriction*: the instrument should be correlated with
    the dependent variable only through the explanatory variable; in our
    case, it should be correlated with $Y_i$ only through its
    correlation with $X_i$.

Let’s say we have found an instrumental variable $Z_i$ for the variable
$X_i$. Then, using an IV analyis implies estimating the following model:
$$
\begin{align}
Y_i &= \alpha_1 + \beta X_i + u_i  \quad \text{(Structural Equation)}\\
X_i &= \alpha_2 + \gamma Z_i + e_i  \quad \text{(First Stage Equation)}
\end{align}
$$

where the two conditions we have seen above imply that:

1.  $\gamma \neq 0$;
2.  $Z_i$ is uncorrelated with $u_i$.

In practice, using an IV analysis often implies using a Two-Stages Least
Square (2SLS) estimator. The two steps of 2SLS are:

1.  Estimate the first stage equation by OLS and obtain the predicted
    value of $X_i$. In this way, we have effectively split $X_i$ into $$
    X_i = \underbrace{\hat{X}_i}_\text{exogenous part} + \underbrace{\hat{e}_i}_\text{endogenous part} 
    $$

where $\hat{X_i} \equiv \hat{\alpha_2} + \hat{\gamma} Z_i$.

1.  Plug $\hat{X_i}$ instead of $X_i$ into the structural equation and
    estimate via OLS. We are then using the “exogenous” part of $X_i$ to
    capture $\beta$.

**Warning**: We can run 2SLS following the steps above, but when we want
to do inference we need to be sure we’re using the true residuals in the
structural equation $\hat{u}_i$. The built-in Stata commands `ivregress`
and `ivreg2` automatically give us the right residuals.

Let’s see how to estimate this in Stata. Once again, we can use our
fictional data set simulating wages of workers in the years 1982-2012 in
a fictional country.

``` {stata}
clear* 
*cd ""
use fake_data, clear
describe, detail
```

In Stata, we can perform IV analysis with a 2SLS estimator by using one
of the following two commands: `ivregress` or `ivreg2`. They have a
similar syntax:

``` stata
ivregress 2sls <Y> (<X> = <Z>)

ivreg2 <Y> (<X> = <Z>)
```

where instead of `<Y>`, `<X>`, and `<Z>`, we write the names of the
corresponding dependent, independent, and instrument variables of our
model.

We now have to choose an IV that can work in our setting. A well-known
example for an instrument for years of schooling is studied by Angrist
and Krueger (1991): they propose using $Z$, the quarter of birth. The
premise behind their IV is that students are required to enter school in
the *year they turn 6* but not necessarily when they are *already* 6
years old, creating a relationship between quarter of birth and
schooling. At the same time, the time of the year one is born shouldn’t
affect one’s earnings aside from its effect on schooling.

Let’s see how to estimate a simple IV in Stata using our data and each
one of the commands `ivregress` and `ivreg2`.

``` {stata}
ivregress 2sls earnings (schooling = quarter_birth)
```

``` {stata}
ivreg2 earnings (schooling = quarter_birth)
```

Both Stata functions give us a standard output: the values of the
coefficients, standard errors, p-values, and 95% confidence intervals.
From the regression output, years of schooling does not seem to have any
effect on earnings. However, before trusting these results, we should
check that the two IV assumptions are met in this case.

Notice that `ivreg2` gives us more details about tests we can perform to
assess whether our instrument is valid. We will talk more about these
tests, especially the weak identification test, in the paragraphs below.

## 17.2 Weak Instrument Test

While we cannot really test for the exclusion restriction, we can check
whether our instrument is relevant. We do that by looking directly at
the coefficients in the first stage.

In Stata, we only need to add the option `first` to get an explicit
output for the first stage.

``` {stata}
ivregress 2sls earnings (schooling = quarter_birth), first
```

``` {stata}
ivreg2 earnings (schooling = quarter_birth), first
```

From both methods, we can see that the IV we have chosen is not relevant
for our explanatory variable $X$: *quarter_birth* is not correlated with
*schooling*. Another indicator of the lack of relevance is given by the
F-statistic reported by Stata in the “Weak Identification test” row: as
a rule of thumb, every time its value is less than 10, the instrument is
not relevant.

Whenever the correlation between $X$ and $Z$ is very close to zero (as
in our case), we say we have a **weak instrument** problem. In practice,
this problem will result in severe finite-sample bias and large variance
in our estimates. Since our instrument is not valid, we cannot trust the
results we have obtained.

## 17.3 Wrap Up

In this module, we studied the linear IV model and how to estimate it
using the 2SLS Method using `ivregress` or `ivreg2`. We learned that we
can overcome the endogeneity problem when we have access to a different
type of variable: an instrumental variable. A good instrument must
satisfy two important conditions:

1.  It must be uncorrelated with the error term (also referred to as the
    exclusion restriction).
2.  It must be correlated, after controlling for observables, with the
    variable of interest (there must be a first stage).

While the second condition can be checked using the regression results
of the first stage, the first condition is inherently not testable.
Therefore, any project that uses IVs must include a discussion, using
contextual knowledge, of why the first condition may hold.

Finally, do not forget that for every endogenous variable in our
regression, we require at least one instrument. For example, if we have
a regression with two endogenous variables, we require at least two IVs!

## 17.4 Wrap-up Table

| Command | Function |
|--------------------------------------|----------------------------------|
| `ivregress 2sls` | It performs Instrumental Variable analysis using a Two-Stage Least Squares estimator. |
| `ivreg2` | It performs Instrumental Variable analysis using a Two-Stage Least Squares estimator by default. |
| `, first` | This option shows the results for the First Stage regression in the IV analysis. |

## References

[Instrumental-variables regression using
Stata](https://www.youtube.com/watch?v=lbnswRJ1qV0)